# Importation des modules utilisés et création des connexions

In [3]:
import sqlite3
import pandas
import pymongo
import mongojoin

In [4]:
# Connection SQL
conn = sqlite3.connect("ClassicModel.sqlite")

# Connection MongDB
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client.SAE_NoSQL

In [70]:
Q1= pandas.read_sql_query("""SELECT t1.customerName
            from Customers as t1
            left join Orders as t2 on t1.customerNumber=t2.customerNumber
            where t2.orderNumber is NULL""",conn)
print(Q1)

                      customerName
0               Havel & Zbyszek Co
1           American Souvenirs Inc
2                Porto Imports Co.
3       Asian Shopping Network, Co
4                  Natürlich Autos
5                    ANG Resellers
6         Messner Shopping Network
7                Franken Gifts, Co
8                BG&E Collectables
9                 Schuyler Imports
10                Der Hund Imports
11       Cramer Spezialitäten, Ltd
12           Asian Treasures, Inc.
13            SAR Distributors, Co
14                 Kommission Auto
15          Lisboa Souveniers, Inc
16           Precious Collectables
17  Stuttgart Collectable Exchange
18        Feuer Online Stores, Inc
19                Warburg Exchange
20             Anton Designs, Ltd.
21             Mit Vergnügen & Co.
22       Kremlin Collectables, Co.
23              Raanan Stores, Inc


In [71]:
Q2= pandas.read_sql_query("""SELECT t1.lastName, t1.firstName,
    Count(distinct t2.customerNumber) as Nombre_client,
    count(distinct t3.orderNumber) as Nombre_commande,
    sum(quantityOrdered*priceEach) as montant_total
            from Employees as t1
            left join Customers as t2 on t1.employeeNumber=t2.salesRepEmployeeNumber
            left join Orders as t3 on t2.customerNumber=t3.customerNumber
            left join OrderDetails as t4 on t3.orderNumber=t4.orderNumber
            group by t1.employeeNumber""",conn)
print(Q2)

     lastName firstName  Nombre_client  Nombre_commande  montant_total
0      Murphy     Diane              0                0            NaN
1   Patterson      Mary              0                0            NaN
2    Firrelli      Jeff              0                0            NaN
3   Patterson   William              0                0            NaN
4      Bondur    Gerard              0                0            NaN
5         Bow   Anthony              0                0            NaN
6    Jennings    Leslie              6               34     1210228.57
7    Thompson    Leslie              6               14      378064.72
8    Firrelli     Julie              6               14      422257.44
9   Patterson     Steve              6               18      565516.73
10      Tseng  Foon Yue              7               17      550395.19
11     Vanauf    George              8               22      725598.76
12     Bondur      Loui              6               20      621798.01
13  He

In [72]:
Q3= pandas.read_sql_query("""SELECT t0.city, t0.country,
    Count(distinct t2.customerNumber) as Nombre_client,
    count(distinct t3.orderNumber) as Nombre_commande,
    sum(quantityOrdered*priceEach) as montant_total,
    count(distinct Case when t0.country !=t2.country then t2.customerNumber end) as Nombre_client_etranger
            from Offices t0
            left join Employees as t1 on t0.officeCode=t1.officeCode
            left join Customers as t2 on t1.employeeNumber=t2.salesRepEmployeeNumber
            left join Orders as t3 on t2.customerNumber=t3.customerNumber
            left join OrderDetails as t4 on t3.orderNumber=t4.orderNumber
            group by t1.officeCode""",conn)
print(Q3)

            city    country  Nombre_client  Nombre_commande  montant_total  \
0  San Francisco        USA             12               48     1588293.29   
1         Boston        USA             12               32      987774.17   
2            NYC        USA             15               39     1275993.95   
3          Paris     France             29              106     3404055.56   
4          Tokyo      Japan              5               16      503957.58   
5         Sydney  Australia             10               38     1281705.83   
6         London         UK             17               47     1604168.80   

   Nombre_client_etranger  
0                       0  
1                       0  
2                       3  
3                      17  
4                       3  
5                       5  
6                      12  


In [74]:

Q4= pandas.read_sql_query("""SELECT t0.productCode, t0.productName,
    Count(distinct t2.customerNumber) as Nombre_client,
    count(distinct t1.orderNumber) as Nombre_commande,
    sum(quantityOrdered) as quatite_total
            from Products t0
            left join OrderDetails as t1 on t0.productCode=t1.productCode
            left join Orders as t2 on t1.orderNumber=t2.orderNumber
            group by t0.productCode""",conn)
print(Q4)

    productCode                            productName  Nombre_client  \
0      S10_1678  1969 Harley Davidson Ultimate Chopper             26   
1      S10_1949               1952 Alpine Renault 1300             27   
2      S10_2016                  1996 Moto Guzzi 1100i             26   
3      S10_4698   2003 Harley-Davidson Eagle Drag Bike             25   
4      S10_4757                    1972 Alfa Romeo GTA             27   
..          ...                                    ...            ...   
105   S700_3505                            The Titanic             22   
106   S700_3962                         The Queen Mary             24   
107   S700_4002              American Airlines: MD-11S             26   
108    S72_1253                       Boeing X-32A JSF             27   
109    S72_3212                             Pont Yacht             24   

     Nombre_commande  quatite_total  
0                 28         1026.0  
1                 28          961.0  
2        

In [73]:
Q5= pandas.read_sql_query("""SELECT t0.country,
    count(distinct t1.orderNumber) as nombre_commande,
    sum(quantityOrdered*priceEach) as montant_total,
    sum(amount) as montant_total_paye
            from Customers t0
            left join Orders as t1 on t0.customerNumber=t1.customerNumber
            left join OrderDetails as t2 on t1.orderNumber=t2.orderNumber
            left join Payments as t3 on t0.customerNumber=t3.customerNumber
            group by t0.country""",conn)
print(Q5)

         country  nombre_commande  montant_total  montant_total_paye
0      Australia               19     2182269.38        2.482541e+07
1        Austria                7      606187.59        4.090982e+06
2        Belgium                7      283705.44        1.931535e+06
3         Canada                7      448157.12        4.487022e+06
4        Denmark                7      781357.50        7.001114e+06
5        Finland                9      988745.73        1.009620e+07
6         France               37     3160296.75        3.141444e+07
7        Germany                7      576293.44        4.971661e+06
8      Hong Kong                2       48784.36        7.805498e+05
9        Ireland                2      115512.86        9.241029e+05
10        Israel                0            NaN                 NaN
11         Italy               10      945208.16        1.324310e+07
12         Japan                6      496898.36        4.837611e+06
13   Netherlands                0 

In [75]:
Q6= pandas.read_sql_query("""SELECT t0.productLine,
                          t3.country,
    count(distinct t1.orderNumber) as nombre_commande
            from Products t0
            left join OrderDetails as t1 on t0.productCode=t1.productCode
            left join Orders as t2 on t1.orderNumber=t2.orderNumber
            left join Customers as t3 on t2.customerNumber=t3.customerNumber
            group by t0.productLine, t3.country""",conn)
print(Q6)

      productLine    country  nombre_commande
0    Classic Cars       None                0
1    Classic Cars  Australia               12
2    Classic Cars    Austria                5
3    Classic Cars    Belgium                2
4    Classic Cars     Canada                6
..            ...        ...              ...
122  Vintage Cars  Singapore                4
123  Vintage Cars      Spain               22
124  Vintage Cars     Sweden                4
125  Vintage Cars         UK               10
126  Vintage Cars        USA               67

[127 rows x 3 columns]


In [76]:
Q7= pandas.read_sql_query("""SELECT t0.productLine,
                          t3.country,
    sum(distinct t4.amount) as montant_total_paye
            from Products t0
            left join OrderDetails as t1 on t0.productCode=t1.productCode
            left join Orders as t2 on t1.orderNumber=t2.orderNumber
            left join Customers as t3 on t2.customerNumber=t3.customerNumber
            left join Payments as t4 on t3.customerNumber=t4.customerNumber
            group by t0.productLine, t3.country""",conn)
print(Q7)

      productLine    country  montant_total_paye
0    Classic Cars       None                 NaN
1    Classic Cars  Australia           566642.90
2    Classic Cars    Austria           134939.48
3    Classic Cars    Belgium           100000.67
4    Classic Cars     Canada           190385.59
..            ...        ...                 ...
122  Vintage Cars  Singapore           285900.83
123  Vintage Cars      Spain          1087510.36
124  Vintage Cars     Sweden           210014.21
125  Vintage Cars         UK           438077.65
126  Vintage Cars        USA          3162135.35

[127 rows x 3 columns]


In [77]:
Q8= pandas.read_sql_query("""SELECT t0.productName,
    avg(priceEach - buyPrice) as marge_moy
            from Products t0
            left join OrderDetails as t1 on t0.productCode=t1.productCode
            group by t0.productCode
            order by marge_moy desc
            limit 10""",conn)
print(Q8)

                            productName  marge_moy
0              1952 Alpine Renault 1300  99.006429
1  2003 Harley-Davidson Eagle Drag Bike  95.235000
2           1992 Ferrari 360 Spider red  83.334906
3                      2002 Suzuki XREO  83.201429
4                1928 Mercedes-Benz SSK  82.696786
5                     2001 Ferrari Enzo  81.043704
6                      1969 Ford Falcon  77.335926
7        1948 Porsche Type 356 Roadster  72.636800
8          1999 Indy 500 Monte Carlo SS  71.794400
9              1917 Grand Touring Sedan  70.432800


In [78]:
Q9= pandas.read_sql_query("""SELECT t0.productName,
                          t3.customerName,t3.customerNumber,
    (priceEach - buyPrice) as marge,
    sum(case when (priceEach - buyPrice) <=0 then 1 else 0 end) as nombre_commande
            from Products t0
            left join OrderDetails as t1 on t0.productCode=t1.productCode
            left join Orders as t2 on t1.orderNumber=t2.orderNumber
            left join Customers as t3 on t2.customerNumber=t3.customerNumber
            where marge <=0
            group by t0.productName""",conn)
print(Q9)

                                productName                  customerName  \
0       18th Century Vintage Horse Carriage  Online Diecast Creations Co.   
1                     18th century schooner          Classic Legends Inc.   
2                         1903 Ford Model A        Euro+ Shopping Channel   
3               1913 Ford Model T Speedster       Anna's Decorations, Ltd   
4                  1917 Grand Touring Sedan  Motor Mint Distributors Inc.   
5          1928 British Royal Navy Airplane       Tokyo Collectables, Ltd   
6                    1928 Mercedes-Benz SSK                AV Stores, Co.   
7                        1934 Ford V8 Coupe  Online Diecast Creations Co.   
8                     1936 Chrysler Airflow    Corrida Auto Replicas, Ltd   
9                      1937 Lincoln Berline            Signal Gift Stores   
10                   1940 Ford Pickup Truck               Mini Wheels Co.   
11                         1940s Ford truck       Anna's Decorations, Ltd   

In [79]:
Q10= pandas.read_sql_query("""SELECT t0.customerName,
    sum(quantityOrdered*priceEach) as montant_total,
    sum(amount) as montant_total_paye
            from Customers t0
            left join Orders as t1 on t0.customerNumber=t1.customerNumber
            left join OrderDetails as t2 on t1.orderNumber=t2.orderNumber
            left join Products as ta on ta.productCode=t2.productCode
            left join Payments as t3 on t0.customerNumber=t3.customerNumber
            group by ta.productCode
            having montant_total_paye > montant_total""",conn)
print(Q10)

                   customerName  montant_total  montant_total_paye
0    Australian Collectors, Co.      385363.80          3803462.17
1    Australian Collectors, Co.      773683.12          4616817.86
2             Atelier graphique      380944.14          3681086.32
3    Australian Collectors, Co.      619126.91          3783800.96
4            Baane Mini Imports      321080.48          3110036.71
..                          ...            ...                 ...
104           La Rochelle Gifts      379448.19          5213490.91
105           La Rochelle Gifts      340031.16          5339307.28
106           La Rochelle Gifts      264140.28          3885156.42
107           Land of Toys Inc.      199570.17          3736749.03
108           La Rochelle Gifts      304568.59          5376226.58

[109 rows x 3 columns]


# Récupération du contenu avec des requêtes SQL

In [12]:
employe = pandas.read_sql_query(
    "SELECT * FROM Employees INNER JOIN Offices USING (officeCode);", 
    conn
)

customers = pandas.read_sql_query(
    "SELECT * FROM Customers;", 
    conn
)

Payments = pandas.read_sql_query(
    "SELECT * FROM Payments;", 
    conn
)

product = pandas.read_sql_query(
    "SELECT * FROM Products;", 
    conn
)

order_detail = pandas.read_sql_query(
    "SELECT * FROM OrderDetails INNER JOIN Products USING (productCode);", 
    conn
)


order = pandas.read_sql_query(
    "SELECT * FROM Orders;", 
    conn
)


In [ ]:
employe = employe.assign()

In [8]:
db.employe.insert_many(
    employe.drop(columns=["officeCode"])
    .to_dict(orient = "records")
)

InsertManyResult([ObjectId('675c4fc7f5c710c5a290e9ec'), ObjectId('675c4fc7f5c710c5a290e9ed'), ObjectId('675c4fc7f5c710c5a290e9ee'), ObjectId('675c4fc7f5c710c5a290e9ef'), ObjectId('675c4fc7f5c710c5a290e9f0'), ObjectId('675c4fc7f5c710c5a290e9f1'), ObjectId('675c4fc7f5c710c5a290e9f2'), ObjectId('675c4fc7f5c710c5a290e9f3'), ObjectId('675c4fc7f5c710c5a290e9f4'), ObjectId('675c4fc7f5c710c5a290e9f5'), ObjectId('675c4fc7f5c710c5a290e9f6'), ObjectId('675c4fc7f5c710c5a290e9f7'), ObjectId('675c4fc7f5c710c5a290e9f8'), ObjectId('675c4fc7f5c710c5a290e9f9'), ObjectId('675c4fc7f5c710c5a290e9fa'), ObjectId('675c4fc7f5c710c5a290e9fb'), ObjectId('675c4fc7f5c710c5a290e9fc'), ObjectId('675c4fc7f5c710c5a290e9fd'), ObjectId('675c4fc7f5c710c5a290e9fe'), ObjectId('675c4fc7f5c710c5a290e9ff'), ObjectId('675c4fc7f5c710c5a290ea00'), ObjectId('675c4fc7f5c710c5a290ea01'), ObjectId('675c4fc7f5c710c5a290ea02')], acknowledged=True)

In [ ]:
Payment = [
    Payments.query('customerNumber== @id')
    .drop(columns=["customerNumber"])
    .to_dict(orient = "records")
    for id in customers.customerNumber
]

customer = customers.assign(payments = Payment)

In [10]:
db.customers.insert_many(
    customer.to_dict(orient = "records")
)

InsertManyResult([ObjectId('675c513df5c710c5a290ea03'), ObjectId('675c513df5c710c5a290ea04'), ObjectId('675c513df5c710c5a290ea05'), ObjectId('675c513df5c710c5a290ea06'), ObjectId('675c513df5c710c5a290ea07'), ObjectId('675c513df5c710c5a290ea08'), ObjectId('675c513df5c710c5a290ea09'), ObjectId('675c513df5c710c5a290ea0a'), ObjectId('675c513df5c710c5a290ea0b'), ObjectId('675c513df5c710c5a290ea0c'), ObjectId('675c513df5c710c5a290ea0d'), ObjectId('675c513df5c710c5a290ea0e'), ObjectId('675c513df5c710c5a290ea0f'), ObjectId('675c513df5c710c5a290ea10'), ObjectId('675c513df5c710c5a290ea11'), ObjectId('675c513df5c710c5a290ea12'), ObjectId('675c513df5c710c5a290ea13'), ObjectId('675c513df5c710c5a290ea14'), ObjectId('675c513df5c710c5a290ea15'), ObjectId('675c513df5c710c5a290ea16'), ObjectId('675c513df5c710c5a290ea17'), ObjectId('675c513df5c710c5a290ea18'), ObjectId('675c513df5c710c5a290ea19'), ObjectId('675c513df5c710c5a290ea1a'), ObjectId('675c513df5c710c5a290ea1b'), ObjectId('675c513df5c710c5a290ea

In [13]:
detail = [
    order_detail.query('orderNumber== @id')
    .drop(columns=["orderNumber","productCode"])
    .to_dict(orient = "records")
    for id in order.orderNumber
]

order = order.assign(details = detail)

,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber,details
0,10100,2003/1/6 0:00:00,2003/1/13 0:00:00,2003/1/10 0:00:00,Shipped,NULL,363,"[{'quantityOrdered': 30, 'priceEach': 171.7, '..."
1,10101,2003/1/9 0:00:00,2003/1/18 0:00:00,2003/1/11 0:00:00,Shipped,Check on availability.,128,"[{'quantityOrdered': 25, 'priceEach': 151.28, ..."
2,10102,2003/1/10 0:00:00,2003/1/18 0:00:00,2003/1/14 0:00:00,Shipped,NULL,181,"[{'quantityOrdered': 39, 'priceEach': 123.29, ..."
3,10103,2003/1/29 0:00:00,2003/2/7 0:00:00,2003/2/2 0:00:00,Shipped,NULL,121,"[{'quantityOrdered': 26, 'priceEach': 207.87, ..."
4,10104,2003/1/31 0:00:00,2003/2/9 0:00:00,2003/2/1 0:00:00,Shipped,NULL,141,"[{'quantityOrdered': 34, 'priceEach': 175.25, ..."


In [14]:
db.orders.insert_many(
    order.to_dict(orient = "records")
)

InsertManyResult([ObjectId('6765af8d26c6ab393b7b3038'), ObjectId('6765af8d26c6ab393b7b3039'), ObjectId('6765af8d26c6ab393b7b303a'), ObjectId('6765af8d26c6ab393b7b303b'), ObjectId('6765af8d26c6ab393b7b303c'), ObjectId('6765af8d26c6ab393b7b303d'), ObjectId('6765af8d26c6ab393b7b303e'), ObjectId('6765af8d26c6ab393b7b303f'), ObjectId('6765af8d26c6ab393b7b3040'), ObjectId('6765af8d26c6ab393b7b3041'), ObjectId('6765af8d26c6ab393b7b3042'), ObjectId('6765af8d26c6ab393b7b3043'), ObjectId('6765af8d26c6ab393b7b3044'), ObjectId('6765af8d26c6ab393b7b3045'), ObjectId('6765af8d26c6ab393b7b3046'), ObjectId('6765af8d26c6ab393b7b3047'), ObjectId('6765af8d26c6ab393b7b3048'), ObjectId('6765af8d26c6ab393b7b3049'), ObjectId('6765af8d26c6ab393b7b304a'), ObjectId('6765af8d26c6ab393b7b304b'), ObjectId('6765af8d26c6ab393b7b304c'), ObjectId('6765af8d26c6ab393b7b304d'), ObjectId('6765af8d26c6ab393b7b304e'), ObjectId('6765af8d26c6ab393b7b304f'), ObjectId('6765af8d26c6ab393b7b3050'), ObjectId('6765af8d26c6ab393b7b30

# Fermeture de la connexion : IMPORTANT à faire dans un cadre professionnel

In [ ]:
conn.close()

# Récupération du contenu avec des rêquetes NoSQL MongoDB

In [5]:
# Requête Q1 : Clients sans commandes
Q1 = db.customers.aggregate([
    {"$lookup": {
        "from": "orders",
        "localField": "customerNumber",
        "foreignField": "customerNumber",
        "as": "orders"
    }},
    {"$match": {"orders": {"$size": 0}}},  # Les clients sans commande
    {"$project": {"_id": 0, "customerName": 1}}
])

pandas.DataFrame(list(Q1))

,customerName
0,Havel & Zbyszek Co
1,American Souvenirs Inc
2,Porto Imports Co.
3,"Asian Shopping Network, Co"
4,Natürlich Autos
5,ANG Resellers
6,Messner Shopping Network
7,"Franken Gifts, Co"
8,BG&E Collectables
9,Schuyler Imports
